# Which SDP solvers should we use?

Julia can call [multiple SDP solvers](https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers). These solvers are broadly classified as:

+ Interior point methods (2nd order methods): CSDP, MOSEK, SeDuMi, SDPT3, DSDP, SDPA
+ First order methods (SCS, ADMM)
+ A bunch of other methods summarized in [wiki](https://en.wikipedia.org/wiki/Semidefinite_programming#Algorithms)

[Here is a detailed benchmark](http://plato.asu.edu/ftp/sparse_sdp.html). The fastest (and successfully solving the most problems) is MOSEK, closely followed by SDPT3 and CSDP, then others. Matteo uses DSDP solver which is slower than those 3 but [it is also available](https://github.com/jump-dev/DSDP.jl) in Julia. 

In [1]:
using Revise
using Knockoffs
using Test
using LinearAlgebra
using Random
using StatsBase
using Statistics
using Distributions
using ToeplitzMatrices
using RCall
using PositiveFactorizations
using UnicodePlots
using MATLAB
using SCS
using JuMP
using Convex

# simulate data
Random.seed!(2022)
n = 100
p = 200
ρ = 0.4
Sigma = Matrix(SymmetricToeplitz(ρ.^(0:(p-1))));

Our model is

$$X_{p \times 1} \sim N(\mathbf{0}_p, \Sigma)$$
where
$$
\Sigma = 
\begin{pmatrix}
    1 & \rho & \rho^2 & ... & \rho^p\\
    \rho & \rho^2 & & ... & \rho^{p-1}\\
    \vdots & & & \rho^2 & \vdots \\
    \rho^p & \cdots & & & 1
\end{pmatrix}
$$
Given $n$ iid samples from the above distribution, we will generate knockoffs according to 
$$(X, \tilde{X}) \sim N
\left(0, \ 
\begin{pmatrix}
    \Sigma & \Sigma - diag(s)\\
    \Sigma - diag(s) & \Sigma
\end{pmatrix}
\right)
$$
where vector $s$ is the solution to

\begin{align}
\text{maximize} & \sum_j |1-s_j|\\
\text{ subject to } & s_{j} \ge 0,\\
  & 2\Sigma - diag(s) \in PSD
\end{align}

## Convex + SCS

In [2]:
# solve SDP using Convex.jl
function julia_sdp(Sigma::Matrix)
    p = size(Sigma, 1)
    svar = Variable(p)
    problem = maximize(sum(svar), svar ≥ 0, 1 ≥ svar, 2*Sigma - Diagonal(svar) == Semidefinite(p))
    solve!(problem, SCS.Optimizer())
    return evaluate(svar)
end
@time s_julia = julia_sdp(Sigma)

131.154706 seconds (63.37 M allocations: 3.791 GiB, 1.08% gc time, 19.92% compilation time)
------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 40201, constraints m: 80401
cones: 	  z: primal zero / dual free vars: 59901
	  l: linear vars: 400
	  s: psd vars: 20100, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, warm_start: 0
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct
	  nnz(A): 100701, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 8.69e+00  9.98e-01  1.69e+03 -1.09e+03 

200-element Vector{Float64}:
 0.9999831968499809
 0.9713101250950568
 0.8111610020741279
 0.8753851680693364
 0.8499617029261299
 0.8604197007608251
 0.8565600229194104
 0.8584213370586803
 0.8579840302238552
 0.8584434210925962
 0.8585231428041151
 0.8587359710406687
 0.858880681212524
 ⋮
 0.8587359710406745
 0.8585231428041343
 0.8584434210926003
 0.8579840302238578
 0.8584213370586931
 0.8565600229194121
 0.8604197007608134
 0.8499617029261346
 0.875385168069348
 0.8111610020741296
 0.9713101250950765
 0.9999831968499767

# Convex + Hypatia

This seems to be the best. 

In [6]:
using Hypatia

function julia_sdp(Σ::Matrix)
    s = Variable(size(Σ,1), Convex.Positive())
    add_constraint!(s, s ≤ 1)
    constraint = 2*Σ - diagm(s) ⪰ 0
    problem = maximize(sum(s),constraint)
    solve!(problem, Hypatia.Optimizer)
    return evaluate(s)  
end

@time s_julia = julia_sdp(Sigma) # compile
@time s_julia = julia_sdp(Sigma) 

19900 of 19901 primal equality constraints are dependent

 iter        p_obj        d_obj |  abs_gap    x_feas    z_feas |      tau       kap        mu | dir_res     prox  step     alpha
    0  -1.0343e+02  -4.8284e+02 | 6.00e+02  1.46e-01  3.31e-01 | 1.00e+00  1.00e+00  1.00e+00 |
    1  -1.2278e+02  -3.9385e+02 | 4.20e+02  1.04e-01  2.37e-01 | 9.80e-01  7.08e-01  7.00e-01 | 5.9e-15  1.9e-01  co-a  3.00e-01
    2  -1.5348e+02  -2.8300e+02 | 2.10e+02  4.99e-02  1.13e-01 | 1.03e+00  3.28e-01  3.50e-01 | 5.7e-14  9.2e-01  co-a  5.00e-01
    3  -1.7491e+02  -2.3908e+02 | 1.05e+02  2.47e-02  5.60e-02 | 1.03e+00  1.70e-01  1.74e-01 | 2.8e-14  7.1e-01  co-a  5.00e-01
    4  -1.7987e+02  -2.2482e+02 | 7.34e+01  1.73e-02  3.93e-02 | 1.03e+00  1.18e-01  1.22e-01 | 2.8e-14  2.2e-01  co-a  3.00e-01
    5  -1.8250e+02  -2.1425e+02 | 5.14e+01  1.22e-02  2.77e-02 | 1.02e+00  8.37e-02  8.56e-02 | 7.1e-15  4.6e-01  co-a  3.00e-01
    6  -1.8176e+02  -2.0445e+02 | 3.60e+01  8.74e-03  1.98e-02 | 1.00e+0

200-element Vector{Float64}:
 0.9999999975421989
 0.9714285562750639
 0.8114286000388236
 0.8754285608972914
 0.8498285802412486
 0.8600685709048966
 0.8559725752335535
 0.8576109732729927
 0.856955614169666
 0.8572177577255956
 0.8571129003675105
 0.8571548432807583
 0.8571380661077549
 ⋮
 0.8571548433098494
 0.8571129003234808
 0.8572177577571737
 0.8569556141700471
 0.8576109732517373
 0.8559725752488924
 0.860068570920087
 0.8498285802136867
 0.8754285609148045
 0.8114286000369729
 0.9714285562700409
 0.9999999975421956

# Convex + SCS + dualization

In [7]:
using Dualization

# solve SDP using Convex.jl
function julia_dual(Sigma::Matrix)
    p = size(Sigma, 1)
    svar = Variable(p)
    problem = maximize(sum(svar), svar ≥ 0, 1 ≥ svar, 2*Sigma - Diagonal(svar) == Semidefinite(p))
    solve!(problem, Dualization.dual_optimizer(SCS.Optimizer))
    return evaluate(svar)
end
@time s_dual = julia_dual(Sigma) # compile
@time s_dual = julia_dual(Sigma)

  9.946430 seconds (2.61 M allocations: 222.876 MiB, 0.69% gc time, 0.29% compilation time)
 12.028088 seconds (2.53 M allocations: 218.782 MiB, 0.58% gc time)
------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 80401, constraints m: 60701
cones: 	  z: primal zero / dual free vars: 40201
	  l: linear vars: 400
	  s: psd vars: 20100, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, warm_start: 0
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct
	  nnz(A): 121201, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
----------------------------------------------

200-element Vector{Float64}:
 0.9999831629523064
 0.9717954065607048
 0.8114913130609391
 0.8756237620958564
 0.8500218563405836
 0.8602819306245469
 0.856259951902612
 0.8580445905725752
 0.8575234851306435
 0.8577796795551333
 0.8575871497479202
 0.8576717044215091
 0.8579081820055083
 ⋮
 0.8576717044215467
 0.8575871497479558
 0.8577796795551594
 0.8575234851306626
 0.8580445905725903
 0.8562599519026469
 0.8602819306245854
 0.8500218563406028
 0.875623762095871
 0.8114913130609658
 0.971795406560729
 0.9999831629523045

# Convex + SCS + dualization

How to tuning convergence tolerance?

In [26]:
using Dualization

# solve SDP using Convex.jl
function julia_dual(Sigma::Matrix)
    p = size(Sigma, 1)
    svar = Variable(p)
    problem = maximize(sum(svar), svar ≥ 0, 1 ≥ svar, 2*Sigma - Diagonal(svar) == Semidefinite(p))
    optimizer = MOI.OptimizerWithAttributes(
        dual_optimizer(SCS.Optimizer),
#         MOI.Silent() => true,
        "max_iters" => 1000,
        "eps_abs" => 1e-3,
        "eps_rel" => 1e-3,

# eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
# 	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
# 	  max_iters: 100000, normalize: 1, warm_start: 0
    )
    
    solve!(problem, optimizer)
    return evaluate(svar)
end
@time s_dual = julia_dual(Sigma) # compile
@time s_dual = julia_dual(Sigma)

  4.560019 seconds (2.76 M allocations: 231.150 MiB, 1.17% gc time, 2.72% compilation time)
  5.599210 seconds (2.53 M allocations: 218.783 MiB, 0.52% gc time)
------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 80401, constraints m: 60701
cones: 	  z: primal zero / dual free vars: 40201
	  l: linear vars: 400
	  s: psd vars: 20100, ssize: 1
settings: eps_abs: 1.0e-03, eps_rel: 1.0e-03, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 1000, normalize: 1, warm_start: 0
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct
	  nnz(A): 121201, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------

200-element Vector{Float64}:
 0.9999556311042131
 0.9750881384892296
 0.8145017591413266
 0.8784169164021525
 0.8527267479196757
 0.8631139759786685
 0.8592457896259493
 0.861115252996954
 0.8608033201623148
 0.861426943331697
 0.8617645084558724
 0.8622315501222684
 0.8626145625308546
 ⋮
 0.8622315501222698
 0.8617645084558754
 0.8614269433316982
 0.8608033201623142
 0.8611152529969568
 0.8592457896259493
 0.8631139759786713
 0.8527267479196776
 0.8784169164021551
 0.8145017591413277
 0.9750881384892308
 0.9999556311042131

# CSDP with JuMP (2nd order method)

In [ ]:
# Build model via JuMP
model = Model(CSDP.Optimizer)
@variable(model, 0 ≤ s[i = 1:p] ≤ 1)
@objective(model, Max, sum(s))
@constraint(model, Symmetric(2Σ - diagm(s[1:p])) in PSDCone());

# Solve optimization problem with ProxSDP
@time JuMP.optimize!(model)

# Retrieve solution
s_csdp = JuMP.value.(s)

# Convex + ProxSDP

In [21]:
using ProxSDP
svar = Variable(p)
problem = maximize(sum(svar), svar ≥ 0, 1 ≥ svar, 2Σ - Diagonal(svar) in :SDP)
@time solve!(problem, () -> ProxSDP.Optimizer())
s_proxSDP = clamp.(evaluate(svar), 0, 1) # make sure s_j ∈ (0, 1)

459.630490 seconds (43.71 M allocations: 17.756 GiB, 0.83% gc time, 4.46% compilation time)


200-element Vector{Float64}:
 0.9993285836369974
 0.9512982065355605
 0.8000111509689896
 0.8597766517160904
 0.834969682320323
 0.8434658379835813
 0.8385505046871335
 0.8392920647925852
 0.8384016140580803
 0.839005161593989
 0.8398529965827887
 0.8410731239904804
 0.8420158680287929
 ⋮
 0.8423242319636166
 0.8413332269440558
 0.840660976773489
 0.8400268290018699
 0.8407573984193294
 0.8397625148779048
 0.8443742169690838
 0.8357115983717658
 0.8608499173665781
 0.8012360129356645
 0.9531163236762759
 0.9993655506420791